# Mask Convolutional Neural Network for Multi-level Hierarchical Classification

In [ ]:
# To be run on Google Colab!
# !mkdir -p deakin/edu/au/
# !wget --directory-prefix=deakin/edu/au/  https://raw.githubusercontent.com/rbouadjenek/Masked-CNN/main/deakin/edu/au/data.py   > /dev/null 2> /dev/null 
# !wget --directory-prefix=deakin/edu/au/ https://raw.githubusercontent.com/rbouadjenek/Masked-CNN/main/deakin/edu/au/metrics.py  > /dev/null 2> /dev/null 
# !wget --directory-prefix=deakin/edu/au/ https://raw.githubusercontent.com/rbouadjenek/Masked-CNN/main/deakin/edu/au/models.py  > /dev/null 2> /dev/null 
# !wget --directory-prefix=deakin/edu/au/ https://raw.githubusercontent.com/rbouadjenek/Masked-CNN/main/deakin/edu/au/__init__.py  > /dev/null 2> /dev/null 
# !wget --directory-prefix=deakin/edu/ https://raw.githubusercontent.com/rbouadjenek/Masked-CNN/main/deakin/edu/au/__init__.py  > /dev/null 2> /dev/null 
# !wget --directory-prefix=deakin/ https://raw.githubusercontent.com/rbouadjenek/Masked-CNN/main/deakin/edu/au/__init__.py  > /dev/null 2> /dev/null 
# !pip install treelib > /dev/null 2> /dev/null 
# !pip install --upgrade scikit-learn > /dev/null 2> /dev/null 


In [ ]:
from deakin.edu.au.data import Cifar100, Stanford_Cars, CU_Birds_200_2011
from deakin.edu.au import models 
import deakin.edu.au.metrics as metrics
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from graphviz import Digraph
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.patches import Rectangle
import numpy as np
from sklearn.metrics import accuracy_score
from keras.callbacks import TensorBoard
from tensorflow.keras.applications import VGG19
import random
from tensorflow import keras
import collections, h5py
from tensorflow.python.keras.saving import hdf5_format


%load_ext autoreload
%autoreload 2
%load_ext tensorboard


# Loading data

## CIFAR100

In [ ]:
#Load dataset
dataset = Cifar100()


## Stanford Cars

In [ ]:
#Load dataset
dataset = Stanford_Cars(image_size=(64,64))


## CU_Birds_200_2011

In [ ]:
#Load dataset
dataset = CU_Birds_200_2011(image_size=(64,64))


In [ ]:
# plot the tree
tree = dataset.get_tree()
tree.show()
# Set variables
batch = 128
epochs = 100
# VGG-19
conv_base = 'vgg19'
learning_rate=1e-5
# NIN
# conv_base = 'nin'
# learning_rate=1e-3

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
performance_callback_val = models.performance_callback(dataset.X_val, 
                                                       dataset.y_val, 
                                                       dataset.taxonomy, 
                                                       dataset.get_tree(),
                                                       name='Validation set')

performance_callback_train = models.performance_callback(dataset.X_train, 
                                                         dataset.y_train, 
                                                         dataset.taxonomy, 
                                                         dataset.get_tree(),
                                                         name='Training set')

checkpoint_filepath = '/tmp/checkpoint_new'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


# Visualization

In [ ]:
###############################################
#### Show distribution of images per class.
###############################################
counter=collections.Counter([x[0] for x in dataset.y_val[0]])
v = [ [dataset.labels[0][item[0]],item[1]]  for item in counter.items()]
df = pd.DataFrame(data=v, columns=['index','value'])
g = sns.catplot(x='index', y= 'value',  data=df, kind='bar', 
                legend=False,height=4,aspect=4,saturation=1)
(g.despine(top=False,right=False))
plt.xlabel("Classes")
plt.ylabel("#images")
plt.title("Distribution of images per class for Level 0")
plt.xticks(rotation='vertical')
plt.show()
###############################################
#### Show distribution of images per class.
###############################################
counter=collections.Counter([x[0] for x in dataset.y_val[1]])
v = [ [dataset.labels[1][item[0]],item[1]]  for item in counter.items()]
df = pd.DataFrame(data=v, columns=['index','value'])
g = sns.catplot(x='index', y= 'value',  data=df, kind='bar', 
                legend=False,height=4,aspect=4,saturation=1)
(g.despine(top=False,right=False))
plt.xlabel("Classes")
plt.ylabel("#images")
plt.title("Distribution of images per class for Level 0")
plt.xticks(rotation='vertical')
plt.show()
###############################################
#### Show distribution of images per class.
###############################################
counter=collections.Counter([x[0] for x in dataset.y_val[2]])
v = [ [dataset.labels[2][item[0]],item[1]]  for item in counter.items()]
df = pd.DataFrame(data=v, columns=['index','value'])
g = sns.catplot(x='index', y= 'value',  data=df, kind='bar', 
                legend=False,height=4,aspect=4,saturation=1)
(g.despine(top=False,right=False))
plt.xlabel("Classes")
plt.ylabel("#images")
plt.title("Distribution of images per class for Level 0")
plt.xticks(rotation='vertical')
plt.show()
###############################################
#### Examples of images.
###############################################

plt.figure(figsize=(20, 16))
for i in range(30):
    k = random.randint(0, len(dataset.X_val))
    file_name = dataset.val_filenames[k]
    y_l0 = dataset.labels[0][dataset.y_val[0][k][0]]
    y_l1 = dataset.labels[1][dataset.y_val[1][k][0]]
    y_l2 = dataset.labels[2][dataset.y_val[2][k][0]]

    ax = plt.subplot(5, 6, i + 1)
    plt.imshow(dataset.X_val[k])
    plt.title(file_name + "\n" + y_l0 + "\n" + y_l1 + "\n" + y_l2)
    plt.axis("off")

plt.tight_layout() 
plt.show()


In [ ]:
def learning_plots(history):
    plt.figure(figsize=(15,4))
    ax1 = plt.subplot(1, 2, 1)
    for l in history.history:
        if l == 'loss' or l == 'val_loss':  
            loss = history.history[l]
            plt.plot(range(1, len(loss) + 1), loss, label=l)

    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    ax2 = plt.subplot(1, 2, 2)
    for k in history.history:
        if 'accuracy' in k:  
            loss = history.history[k]
            plt.plot(range(1, len(loss) + 1), loss, label=k)
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


# Training all models

In [ ]:
# Create the model
mnets_model = models.get_mnets(dataset.num_classes, 
                               dataset.image_size, 
                               conv_base=conv_base,
                               learning_rate=learning_rate)
mnets_model.summary()
#train model
history_mnets_model = mnets_model.fit(dataset.X_train, 
                                      dataset.y_train,
                                      validation_data = (dataset.X_val, dataset.y_val), 
                                      batch_size=batch, 
                                      epochs=epochs,
                                      callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_mnets_model)
mnets_model.save("models/mnets_model_" + dataset.name + "_" + conv_base)
plot_model(mnets_model)


# with h5py.File("models/mnets_model_" + dataset.name + "_" + conv_base + "model.h5", mode='w') as f:
#     hdf5_format.save_model_to_hdf5(mnets_model, f)



In [ ]:
mnets_model_pred = mnets_model.predict(dataset.X_test)
mnets_model_accuracy = metrics.get_top_k_taxonomical_accuracy(dataset.y_test, mnets_model_pred)
mnets_model_accuracy = [x * 100 for x in mnets_model_accuracy]


In [ ]:
def get_potential_improvement(true_y, pred_y, taxo):
    
    pred_y = [np.argmax(x, axis=1) for x in pred_y]
    oracle_accuracy1 = [0] * len(pred_y)
    oracle_accuracy2 = [0] * len(pred_y)
    oracle_accuracy = [0] * len(pred_y)
    for i in range(len(pred_y)):
        for j in range(len(pred_y[0])):
            # check correctness
            if pred_y[i][j] == true_y[i][j][0]:
                continue
            else:
                # go left to right
                stop = False
                for z in range(i+1, len(pred_y)):
                    if pred_y[z][j] == true_y[z][j][0]:
                        oracle_accuracy[i] = oracle_accuracy[i] + 1
                        oracle_accuracy1[i] = oracle_accuracy1[i] + 1
                        stop = True
                        break
                #go right to left -- do not touch, complex code 
                if stop == False and i > 0:
                    parents = []
                    current = pred_y[i][j]
                    for z in reversed(range(i)):
                        m = taxo[z]
                        row = list(np.transpose(m)[current])
                        parent = row.index(1)
                        current = parent
                        parents.insert(0, parent)
                    for z in reversed(range(i)):
                        if pred_y[z][j] == true_y[z][j][0] and true_y[z][j][0] != parents[z]:
                            oracle_accuracy[i] = oracle_accuracy[i] + 1
                            oracle_accuracy2[i] = oracle_accuracy2[i] + 1
                            break
    
    print("Total improvement: ",oracle_accuracy)
    print("Left to right improvement: ",oracle_accuracy1)
    print("Right to left improvement: ",oracle_accuracy2)
    oracle_accuracy  = [x*100/len(mnets_model_pred[0]) for x in oracle_accuracy] 
    return oracle_accuracy


# Testings
# taxo = [[[1, 1, 0, 0, 0], [0, 0, 1, 1, 1]],
#             [[1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1]]
#             ]
# pred = [np.array([0]), np.array([1]),np.array([2])]
# true_y = [np.array([[0]]), np.array([[0]]),np.array([[0]])]
# get_potential_improvement(pred, true_y, taxo)

# oracle_accuracy = get_potential_improvement(mnets_model_pred,
#                                             dataset.y_test, 
#                                             dataset.taxonomy)



In [ ]:
oracle_accuracy = get_potential_improvement(dataset.y_test, 
                                            mnets_model_pred,
                                            dataset.taxonomy)

labels = ['L0', 'L1', 'L2']
width  = 0.6
params = {'legend.fontsize': 10,
          'axes.labelsize': 18,
          'axes.titlesize': 11,
          'xtick.labelsize': 14,
          'ytick.labelsize': 14,
          'axes.titlepad': 12}
plt.rcParams.update(params)

fig, ax = plt.subplots(figsize=(5,3))
ax.set_axisbelow(True)
ax.bar(labels, mnets_model_accuracy, width, edgecolor = 'black', linewidth = 0.5, color=['#FF3400','#0025FF','#008000','#FFFD07'])
ax.bar(labels, oracle_accuracy, width,  bottom=mnets_model_accuracy, edgecolor = 'black', linewidth = 0.5, color=['#FFC0CB','#ADD8E6','#90EE90','#FFFEE0'])
ax.set_ylabel('Fraction %')
rects = ax.patches
labels = ["+%.2f" % i for i in oracle_accuracy]
labels = [x+"%" for x in labels]

heights = []
for imp, accuracy in zip(oracle_accuracy, mnets_model_accuracy):
    heights.append(accuracy+imp/2)
 

  
for rect, label, height in zip(rects, labels,heights):
    ax.text(rect.get_x() + rect.get_width() / 2, height-2, label,
            ha='center', va='bottom')

plt.yticks([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
# ax.set_yticks([0,2,4,6])
ax.set_ylim(0, 60)
plt.grid(color = 'black', linestyle = '--', linewidth = 0.5)

plt.savefig('plots/oracle_' + dataset.name + '_' + conv_base + '.eps', bbox_inches = 'tight')
plt.show()

In [ ]:
# Create the model
mout_model = models.get_mout_model(dataset.num_classes, 
                                   dataset.image_size, 
                                   conv_base=conv_base,
                                   learning_rate=learning_rate)
mout_model.summary()
#train model
history_mout_model = mout_model.fit(dataset.X_train, 
                                    dataset.y_train,
                                    validation_data = (dataset.X_val, dataset.y_val),
                                    batch_size=batch, 
                                    epochs=epochs,
                                    callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_mout_model)
mout_model.save("models/mout_model_" + dataset.name + "_" + conv_base)
plot_model(mout_model)


In [ ]:
# Create the model
BCNN1_model = models.get_BCNN1(dataset.num_classes, 
                               dataset.image_size, 
                               conv_base=conv_base,
                               learning_rate=learning_rate)
BCNN1_model.summary()
#train model
history_BCNN1_model = BCNN1_model.fit(dataset.X_train, 
                                      dataset.y_train,
                                      validation_data = (dataset.X_val, dataset.y_val),
                                      batch_size=batch, 
                                      epochs=epochs,
                                      callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_BCNN1_model)
BCNN1_model.save("models/BCNN1_model_" + dataset.name + "_" + conv_base)
plot_model(BCNN1_model)


In [ ]:
# Create the model
BCNN1_reversed_model = models.get_BCNN1(dataset.num_classes, 
                                        dataset.image_size, 
                                        reverse=True, 
                                        conv_base=conv_base, 
                                        learning_rate=learning_rate)
BCNN1_reversed_model.summary()
#train model
history_BCNN1_reversed_model = BCNN1_reversed_model.fit(dataset.X_train, 
                                                        dataset.y_train,
                                                        validation_data = (dataset.X_val, dataset.y_val),
                                                        batch_size=batch, 
                                                        epochs=epochs,
                                                        callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_BCNN1_reversed_model)
BCNN1_reversed_model.save("models/BCNN1_reversed_model_" + dataset.name + "_" + conv_base)
plot_model(BCNN1_reversed_model)


In [ ]:
# Create the model
BCNN2_model = models.get_BCNN2(dataset.num_classes, 
                               dataset.image_size, 
                               conv_base=conv_base, 
                               learning_rate=learning_rate)
BCNN2_model.summary()
#train model
history_BCNN2_model = BCNN2_model.fit(dataset.X_train, 
                                      dataset.y_train,
                                      validation_data = (dataset.X_val, dataset.y_val), 
                                      batch_size=batch,
                                      epochs=epochs,
                                      callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_BCNN2_model)
BCNN2_model.save("models/BCNN2_model_" + dataset.name + "_" + conv_base)
plot_model(BCNN2_model)


In [ ]:
# Create the model
BCNN2_reversed_model = models.get_BCNN2(dataset.num_classes, 
                                        dataset.image_size, 
                                        reverse=True, 
                                        conv_base=conv_base,
                                        learning_rate=learning_rate)
BCNN2_reversed_model.summary()
#train model
history_BCNN2_reversed_model = BCNN2_reversed_model.fit(dataset.X_train, 
                                                        dataset.y_train,
                                                        validation_data = (dataset.X_val, dataset.y_val), 
                                                        batch_size=batch, 
                                                        epochs=epochs,
                                                        callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_BCNN2_reversed_model)
BCNN2_reversed_model.save("models/BCNN2_reversed_model_" + dataset.name + "_" + conv_base)
plot_model(BCNN2_reversed_model)


In [ ]:
# Create the model
MLPH_model = models.get_MLPH_model(dataset.num_classes, 
                                   dataset.image_size)
MLPH_model.summary()
#train model
history_MLPH_model = MLPH_model.fit(dataset.X_train, 
                                    dataset.y_train,
                                    validation_data = (dataset.X_val, dataset.y_val), 
                                    batch_size=batch, 
                                    epochs=epochs,
                                    callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_MLPH_model)
MLPH_model.save("models/MLPH_model_" + dataset.name + "_" + conv_base)
plot_model(MLPH_model,show_shapes=False, expand_nested=False, dpi=96)


In [ ]:
# Create the model
baseline_model = models.get_Baseline_model(dataset.num_classes, 
                                           dataset.image_size, 
                                           dataset.taxonomy, 
                                           conv_base=conv_base,
                                           learning_rate=learning_rate)
baseline_model.summary()
#train model
history_baseline_model = baseline_model.fit(dataset.X_train, 
                    dataset.y_train[-1],
                    validation_data = (dataset.X_val, dataset.y_val[-1]),
                    batch_size=batch, 
                    epochs=epochs,
                    callbacks=[performance_callback_val, early_stopping_callback])
learning_plots(history_baseline_model)
baseline_model.save("models/baseline_model_" + dataset.name + "_" + conv_base)
plot_model(baseline_model)


In [ ]:
# tensorboard = TensorBoard(log_dir='/tmp/logs', histogram_freq=0,
#                           write_graph=True, write_images=False)
# Create the model
mcnn_model = models.get_Masked_Output_Net(dataset.num_classes,
                                          dataset.image_size,
                                          dataset.taxonomy,
                                          conv_base=conv_base,
                                          learning_rate=learning_rate)
mcnn_model.summary(line_length=110)
#train model
history_mcnn_model_model = mcnn_model.fit(dataset.X_train, 
                                          dataset.y_train,
                                          validation_data = (dataset.X_val, dataset.y_val),
                                          batch_size=batch, 
                                          epochs=epochs,
                                          callbacks=[performance_callback_train,
                                                     performance_callback_val, 
#                                                      tensorboard, 
                                                     early_stopping_callback
                                                    ])
learning_plots(history_mcnn_model_model)
mcnn_model.save("models/mcnn_model_" + dataset.name + "_" + conv_base)
plot_model(mcnn_model)


In [ ]:
# Create the model
mcnn_mnets_model = models.get_Masked_Output_Net(dataset.num_classes,
                                                dataset.image_size,
                                                dataset.taxonomy,
                                                conv_base=conv_base,
                                                learning_rate=learning_rate,
                                                mnets=True)
mcnn_mnets_model.summary(line_length=110)
#train model
history_mcnn_mnets_model = mcnn_mnets_model.fit(dataset.X_train, 
                                                dataset.y_train,
                                                validation_data=(dataset.X_val, dataset.y_val), 
                                                batch_size=batch, 
                                                epochs=epochs,
                                                callbacks=[performance_callback_train,
                                                           performance_callback_val,  
#                                                            tensorboard, 
                                                           early_stopping_callback
                                                          ])
learning_plots(history_mcnn_mnets_model)
mcnn_mnets_model.save("models/mcnn_mnets_model_" + dataset.name + "_" + conv_base)
plot_model(mcnn_mnets_model)


In [ ]:
mcnn_model_pred = mcnn_model.predict(dataset.X_test)
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=2))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=3))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=4))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=5))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=6))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=7))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=8))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=9))
print(metrics.get_h_accuracy(dataset.y_test, mcnn_model_pred, k=10))


In [ ]:
BCNN2_reversed_pred = BCNN2_reversed_model.predict(dataset.X_test)
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=2))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=3))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=4))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=5))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=6))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=7))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=8))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=9))
print(metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=10))


In [ ]:
# Make predictions on the test set
mout_pred = mout_model.predict(dataset.X_test)
BCNN1_pred = BCNN1_model.predict(dataset.X_test)
BCNN1_reversed_pred = BCNN1_reversed_model.predict(dataset.X_test)
BCNN2_pred = BCNN2_model.predict(dataset.X_test)
BCNN2_reversed_pred = BCNN2_reversed_model.predict(dataset.X_test)
mnets_pred = mnets_model.predict(dataset.X_test)
baseline_pred = baseline_model.predict(dataset.X_test)
MLPH_pred = MLPH_model.predict(dataset.X_test)
mcnn_pred = mcnn_model.predict(dataset.X_test)


In [ ]:
exact_match_scores = [metrics.get_exact_match(dataset.y_test, mout_pred),
                      metrics.get_exact_match(dataset.y_test, BCNN1_pred),
                      metrics.get_exact_match(dataset.y_test, BCNN1_reversed_pred),
                      metrics.get_exact_match(dataset.y_test, BCNN2_pred),
                      metrics.get_exact_match(dataset.y_test, BCNN2_reversed_pred),
                      metrics.get_exact_match(dataset.y_test, mnets_pred),
                      metrics.get_exact_match(dataset.y_test, baseline_pred),
                      metrics.get_exact_match(dataset.y_test, MLPH_pred),
                      metrics.get_exact_match(dataset.y_test, mcnn_pred)]

h_accuracy_scores = [metrics.get_h_accuracy(dataset.y_test, mout_pred),
                     metrics.get_h_accuracy(dataset.y_test, BCNN1_pred),
                     metrics.get_h_accuracy(dataset.y_test, BCNN1_reversed_pred),
                     metrics.get_h_accuracy(dataset.y_test, BCNN2_pred),
                     metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred),
                     metrics.get_h_accuracy(dataset.y_test, mnets_pred),
                     metrics.get_h_accuracy(dataset.y_test, baseline_pred), 
                     metrics.get_h_accuracy(dataset.y_test, MLPH_pred), 
                     metrics.get_h_accuracy(dataset.y_test, mcnn_pred)]

h_accuracy_scores_k2 = [metrics.get_h_accuracy(dataset.y_test, mout_pred, k=2),
                     metrics.get_h_accuracy(dataset.y_test, BCNN1_pred, k=2),
                     metrics.get_h_accuracy(dataset.y_test, BCNN1_reversed_pred, k=2),
                     metrics.get_h_accuracy(dataset.y_test, BCNN2_pred, k=2),
                     metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=2),
                     metrics.get_h_accuracy(dataset.y_test, mnets_pred, k=2),
                     metrics.get_h_accuracy(dataset.y_test, baseline_pred, k=2), 
                     metrics.get_h_accuracy(dataset.y_test, MLPH_pred, k=2), 
                     metrics.get_h_accuracy(dataset.y_test, mcnn_pred, k=2)]

h_accuracy_scores_k5 = [metrics.get_h_accuracy(dataset.y_test, mout_pred, k=5),
                     metrics.get_h_accuracy(dataset.y_test, BCNN1_pred, k=5),
                     metrics.get_h_accuracy(dataset.y_test, BCNN1_reversed_pred, k=5),
                     metrics.get_h_accuracy(dataset.y_test, BCNN2_pred, k=5),
                     metrics.get_h_accuracy(dataset.y_test, BCNN2_reversed_pred, k=5),
                     metrics.get_h_accuracy(dataset.y_test, mnets_pred, k=5),
                     metrics.get_h_accuracy(dataset.y_test, baseline_pred, k=5), 
                     metrics.get_h_accuracy(dataset.y_test, MLPH_pred, k=5), 
                     metrics.get_h_accuracy(dataset.y_test, mcnn_pred, k=5)]

m_accuracy_scores = [metrics.get_m_accuracy(dataset.y_test, mout_pred),
                     metrics.get_m_accuracy(dataset.y_test, BCNN1_pred),
                     metrics.get_m_accuracy(dataset.y_test, BCNN1_reversed_pred),
                     metrics.get_m_accuracy(dataset.y_test, BCNN2_pred),
                     metrics.get_m_accuracy(dataset.y_test, BCNN2_reversed_pred),
                     metrics.get_m_accuracy(dataset.y_test, mnets_pred),
                     metrics.get_m_accuracy(dataset.y_test, baseline_pred),
                     metrics.get_m_accuracy(dataset.y_test, MLPH_pred),
                     metrics.get_m_accuracy(dataset.y_test, mcnn_pred)]


consistency = [metrics.get_consistency(mout_pred, dataset.taxonomy),
               metrics.get_consistency(BCNN1_pred, dataset.taxonomy),
               metrics.get_consistency(BCNN1_reversed_pred, dataset.taxonomy),
               metrics.get_consistency(BCNN2_pred, dataset.taxonomy),
               metrics.get_consistency(BCNN2_reversed_pred, dataset.taxonomy),
               metrics.get_consistency(mnets_pred, dataset.taxonomy),
               metrics.get_consistency(baseline_pred, dataset.taxonomy),
               metrics.get_consistency(MLPH_pred, dataset.taxonomy),
               metrics.get_consistency(mcnn_pred, dataset.taxonomy)]

# Create example dataframe
df = pd.DataFrame({
'exact_match': exact_match_scores,
'h_accuracy': h_accuracy_scores,
'h_accuracy_k2': h_accuracy_scores_k2,
'h_accuracy_k5': h_accuracy_scores_k5,
'm_accuracy': m_accuracy_scores,
'consistency': consistency,
's': [3000,3000,3000,3000,3000,3000,3000,3000,3000],
'algo': ['mout','BCNN1','BCNN1_R','BCNN2','BCNN2_R', 'mnets', 'baseline', 'MLPH', 'M-CNN']
})


In [ ]:
plt.figure(figsize=(6.5, 4))
params = {'legend.fontsize': 18,
          'axes.labelsize': 20,
          'axes.titlesize': 14,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'axes.titlepad': 25,
          'font.size': 10}
plt.rcParams.update(params)

ax = sns.scatterplot(df.exact_match, df.consistency, alpha = 0.5,s = df.s)

ax.grid(color='black', linestyle='--', linewidth=1)
ax.set_axisbelow(True)

plt.xlabel("")
plt.ylabel("consistency")
# plt.title("Box plot of #candidates per cycle and system")
plt.yticks([0.2,0.4,0.6,0.8,1.0])
plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0])

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
plt.xlabel('Exact Match')
plt.ylabel('Consistency')
#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
    if df.algo[line]=='baseline':
        ax.text(df.exact_match[line], df.consistency[line], df.algo[line], verticalalignment='top', horizontalalignment='center', size='medium', color='black', weight='semibold')
    elif df.algo[line].startswith('BCNN'):
        ax.text(df.exact_match[line], df.consistency[line], df.algo[line], horizontalalignment='left', size='medium', color='black', weight='semibold')
    else:
        ax.text(df.exact_match[line], df.consistency[line], df.algo[line], horizontalalignment='center', size='medium', color='black', weight='semibold')



currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((0., 0.), 1., 0.2, color = 'red', zorder=0))
currentAxis.add_patch(Rectangle((0., 0.), .2, 1., color = 'red',zorder=0))
currentAxis.add_patch(Rectangle((0.2, 0.2), 0.2, 0.2, color = 'red', zorder=0))

currentAxis.add_patch(Rectangle((0.2, 0.4), 0.4, 1.0, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.4, 0.2), 1., 0.4, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.6), 0.2, 0.2, color = 'orange', zorder=0))

currentAxis.add_patch(Rectangle((0.8, 0.6), 0.2, 0.4, color = 'green', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.8), 0.4, 0.2, color = 'green', zorder=0))
# plotlim = plt.xlim() + plt.ylim()
# import matplotlib.colors

# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])

# ax.imshow([[0.5, 0.5, 0.5], [0, 0.5, 0.5], [0, 0, 0.5]],
#           cmap=cmap,
#           interpolation='bicubic',
#           extent=plotlim, vmin=0, vmax=0.5)
        
plt.savefig('plots/bubble_plot_em.pdf', bbox_inches = 'tight')
plt.show()


In [ ]:
plt.figure(figsize=(6.5, 4))
params = {'legend.fontsize': 18,
          'axes.labelsize': 20,
          'axes.titlesize': 14,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'axes.titlepad': 25,
          'font.size': 10}
plt.rcParams.update(params)

ax = sns.scatterplot(df.h_accuracy, df.consistency, alpha = 0.5,s = df.s)

ax.grid(color='black', linestyle='--', linewidth=1)
ax.set_axisbelow(True)

plt.xlabel("")
plt.ylabel("consistency")
# plt.title("Box plot of #candidates per cycle and system")
plt.yticks([0.2,0.4,0.6,0.8,1.0])
plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0])

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
plt.xlabel('Harmonic Accuracy')
plt.ylabel('Consistency')
#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
    if df.algo[line]=='baseline':
        ax.text(df.h_accuracy[line], df.consistency[line], df.algo[line], verticalalignment='top', horizontalalignment='center', size='medium', color='black', weight='semibold')
    elif df.algo[line].startswith('BCNN'):
        ax.text(df.h_accuracy[line], df.consistency[line], df.algo[line], horizontalalignment='left', size='medium', color='black', weight='semibold')
    else:
        ax.text(df.h_accuracy[line], df.consistency[line], df.algo[line], horizontalalignment='center', size='medium', color='black', weight='semibold')



currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((0., 0.), 1., 0.2, color = 'red', zorder=0))
currentAxis.add_patch(Rectangle((0., 0.), .2, 1., color = 'red',zorder=0))
currentAxis.add_patch(Rectangle((0.2, 0.2), 0.2, 0.2, color = 'red', zorder=0))

currentAxis.add_patch(Rectangle((0.2, 0.4), 0.4, 1.0, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.4, 0.2), 1., 0.4, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.6), 0.2, 0.2, color = 'orange', zorder=0))

currentAxis.add_patch(Rectangle((0.8, 0.6), 0.2, 0.4, color = 'green', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.8), 0.4, 0.2, color = 'green', zorder=0))
# plotlim = plt.xlim() + plt.ylim()
# import matplotlib.colors

# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])

# ax.imshow([[0.5, 0.5, 0.5], [0, 0.5, 0.5], [0, 0, 0.5]],
#           cmap=cmap,
#           interpolation='bicubic',
#           extent=plotlim, vmin=0, vmax=0.5)
        
plt.savefig('plots/bubble_plot_h_accuracy.pdf', bbox_inches = 'tight')
plt.show()


In [ ]:
plt.figure(figsize=(6.5, 4))
params = {'legend.fontsize': 18,
          'axes.labelsize': 20,
          'axes.titlesize': 14,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'axes.titlepad': 25,
          'font.size': 10}
plt.rcParams.update(params)

ax = sns.scatterplot(df.h_accuracy_k2, df.consistency, alpha = 0.5,s = df.s)

ax.grid(color='black', linestyle='--', linewidth=1)
ax.set_axisbelow(True)

plt.xlabel("")
plt.ylabel("consistency")
# plt.title("Box plot of #candidates per cycle and system")
plt.yticks([0.2,0.4,0.6,0.8,1.0])
plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0])

ax.set_xlim(0,1)
ax.set_ylim(0, 1)
plt.xlabel('Harmonic Accuracy k=2')
plt.ylabel('Consistency')
#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
    if df.algo[line]=='baseline':
        ax.text(df.h_accuracy_k2[line], df.consistency[line], df.algo[line], verticalalignment='top', horizontalalignment='center', size='medium', color='black', weight='semibold')
    elif df.algo[line].startswith('BCNN'):
        ax.text(df.h_accuracy_k2[line], df.consistency[line], df.algo[line], horizontalalignment='left', size='medium', color='black', weight='semibold')
    else:
        ax.text(df.h_accuracy_k2[line], df.consistency[line], df.algo[line], horizontalalignment='center', size='medium', color='black', weight='semibold')



currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((0., 0.), 1., 0.2, color = 'red', zorder=0))
currentAxis.add_patch(Rectangle((0., 0.), .2, 1., color = 'red',zorder=0))
currentAxis.add_patch(Rectangle((0.2, 0.2), 0.2, 0.2, color = 'red', zorder=0))

currentAxis.add_patch(Rectangle((0.2, 0.4), 0.4, 1.0, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.4, 0.2), 1., 0.4, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.6), 0.2, 0.2, color = 'orange', zorder=0))

currentAxis.add_patch(Rectangle((0.8, 0.6), 0.2, 0.4, color = 'green', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.8), 0.4, 0.2, color = 'green', zorder=0))
# plotlim = plt.xlim() + plt.ylim()
# import matplotlib.colors

# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])

# ax.imshow([[0.5, 0.5, 0.5], [0, 0.5, 0.5], [0, 0, 0.5]],
#           cmap=cmap,
#           interpolation='bicubic',
#           extent=plotlim, vmin=0, vmax=0.5)
        
plt.savefig('plots/bubble_plot_h_accuracy_k2.pdf', bbox_inches = 'tight')
plt.show()


In [ ]:
plt.figure(figsize=(6.5, 4))
params = {'legend.fontsize': 18,
          'axes.labelsize': 20,
          'axes.titlesize': 14,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'axes.titlepad': 25,
          'font.size': 10}
plt.rcParams.update(params)

ax = sns.scatterplot(df.h_accuracy_k5, df.consistency, alpha = 0.5,s = df.s)

ax.grid(color='black', linestyle='--', linewidth=1)
ax.set_axisbelow(True)

plt.xlabel("")
plt.ylabel("consistency")
# plt.title("Box plot of #candidates per cycle and system")
plt.yticks([0.2,0.4,0.6,0.8,1.0])
plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0])

ax.set_xlim(0,1)
ax.set_ylim(0, 1)
plt.xlabel('Harmonic Accuracy k=5')
plt.ylabel('Consistency')
#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
    if df.algo[line]=='baseline':
        ax.text(df.h_accuracy_k5[line], df.consistency[line], df.algo[line], verticalalignment='top', horizontalalignment='center', size='medium', color='black', weight='semibold')
    elif df.algo[line].startswith('BCNN'):
        ax.text(df.h_accuracy_k5[line], df.consistency[line], df.algo[line], horizontalalignment='left', size='medium', color='black', weight='semibold')
    else:
        ax.text(df.h_accuracy_k5[line], df.consistency[line], df.algo[line], horizontalalignment='center', size='medium', color='black', weight='semibold')



currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((0., 0.), 1., 0.2, color = 'red', zorder=0))
currentAxis.add_patch(Rectangle((0., 0.), .2, 1., color = 'red',zorder=0))
currentAxis.add_patch(Rectangle((0.2, 0.2), 0.2, 0.2, color = 'red', zorder=0))

currentAxis.add_patch(Rectangle((0.2, 0.4), 0.4, 1.0, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.4, 0.2), 1., 0.4, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.6), 0.2, 0.2, color = 'orange', zorder=0))

currentAxis.add_patch(Rectangle((0.8, 0.6), 0.2, 0.4, color = 'green', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.8), 0.4, 0.2, color = 'green', zorder=0))
# plotlim = plt.xlim() + plt.ylim()
# import matplotlib.colors

# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])

# ax.imshow([[0.5, 0.5, 0.5], [0, 0.5, 0.5], [0, 0, 0.5]],
#           cmap=cmap,
#           interpolation='bicubic',
#           extent=plotlim, vmin=0, vmax=0.5)
        
plt.savefig('plots/bubble_plot_h_accuracy_k5.pdf', bbox_inches = 'tight')
plt.show()


In [ ]:
plt.figure(figsize=(6.5, 4))
params = {'legend.fontsize': 18,
          'axes.labelsize': 20,
          'axes.titlesize': 14,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'axes.titlepad': 25,
          'font.size': 10}
plt.rcParams.update(params)

ax = sns.scatterplot(df.m_accuracy, df.consistency, alpha = 0.5,s = df.s)

ax.grid(color='black', linestyle='--', linewidth=1)
ax.set_axisbelow(True)

plt.xlabel("")
plt.ylabel("consistency")
# plt.title("Box plot of #candidates per cycle and system")
plt.yticks([0.2,0.4,0.6,0.8,1.0])
plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0])

ax.set_xlim(0,1)
ax.set_ylim(0, 1)
plt.xlabel('Mean Accuracy')
plt.ylabel('Consistency')
#For each point, we add a text inside the bubble
for line in range(0,df.shape[0]):
    if df.algo[line]=='baseline':
        ax.text(df.m_accuracy[line], df.consistency[line], df.algo[line], verticalalignment='top', horizontalalignment='center', size='medium', color='black', weight='semibold')
    elif df.algo[line].startswith('BCNN'):
        ax.text(df.m_accuracy[line], df.consistency[line], df.algo[line], horizontalalignment='left', size='medium', color='black', weight='semibold')
    else:
        ax.text(df.m_accuracy[line], df.consistency[line], df.algo[line], horizontalalignment='center', size='medium', color='black', weight='semibold')



currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((0., 0.), 1., 0.2, color = 'red', zorder=0))
currentAxis.add_patch(Rectangle((0., 0.), .2, 1., color = 'red',zorder=0))
currentAxis.add_patch(Rectangle((0.2, 0.2), 0.2, 0.2, color = 'red', zorder=0))

currentAxis.add_patch(Rectangle((0.2, 0.4), 0.4, 1.0, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.4, 0.2), 1., 0.4, color = 'orange', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.6), 0.2, 0.2, color = 'orange', zorder=0))

currentAxis.add_patch(Rectangle((0.8, 0.6), 0.2, 0.4, color = 'green', zorder=0))
currentAxis.add_patch(Rectangle((0.6, 0.8), 0.4, 0.2, color = 'green', zorder=0))
# plotlim = plt.xlim() + plt.ylim()
# import matplotlib.colors

# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])

# ax.imshow([[0.5, 0.5, 0.5], [0, 0.5, 0.5], [0, 0, 0.5]],
#           cmap=cmap,
#           interpolation='bicubic',
#           extent=plotlim, vmin=0, vmax=0.5)
        
plt.savefig('plots/bubble_plot_m_accuracy.pdf', bbox_inches = 'tight')
plt.show()


# This section is used for debugging

In [ ]:
# Create the model
i = 2
baseline_model = m.get_Classifier_model(num_classes[i], 
                                           dataset.image_size, 
                                           conv_base='vgg19',
                                           learning_rate=1e-5,
                                            lam=0.0001)
baseline_model.summary()
#train model
history_baseline_model = baseline_model.fit(dataset.X_train, 
                    dataset.y_train[i],
                    validation_data = (dataset.X_val, dataset.y_val[i]),
                    batch_size=batch, 
                    epochs=200,
                    )
plot_model(baseline_model)


In [ ]:
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense, Activation, Lambda, Conv2D, MaxPool2D, \
    GlobalAveragePooling2D, Multiply, Concatenate, experimental
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19, VGG16, ResNet50, Xception


i = 2
in_layer = Input(shape=dataset.image_size)
rescale = experimental.preprocessing.Rescaling(1. / 255)(in_layer)
conv_base = VGG19(include_top=False, weights="imagenet")(rescale)
conv_base = Flatten()(conv_base)
# create output layers
out_layer = Dense(num_classes[i], activation="softmax")(conv_base)
# Build the model
model = Model(inputs=in_layer,
              outputs=out_layer)
loss = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])
model.summary()


history_baseline_model = model.fit(dataset.X_val, 
                    dataset.y_val[i],
                    validation_data = (dataset.X_val, dataset.y_val[i]),
                    batch_size=batch, 
                    epochs=200,
                    )
plot_model(baseline_model)


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score, classification_report
from tensorflow.keras import models, layers, optimizers
from tensorflow.python.keras.saving import hdf5_format
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
import h5py, itertools, collections
import itertools
from keras.applications.efficientnet import EfficientNetB0


In [ ]:
# Defining your model here:
i = 2
model = models.Sequential()
model.add(keras.Input(shape=dataset.image_size)) 
model.add(layers.experimental.preprocessing.Rescaling(1./255))
model.add(EfficientNetB0(weights='imagenet', include_top=False))
#Dense part
model.add(layers.Flatten())
model.add(layers.Dense(num_classes[i], activation='softmax'))
# Print a summary of the model
loss = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

model.summary()

history_baseline_model = model.fit(dataset.X_train, 
                    dataset.y_train[i],
                    validation_data = (dataset.X_val, dataset.y_val[i]),
                    batch_size=batch, 
                    epochs=200,
                    )
plot_model(baseline_model)


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score, classification_report
from tensorflow.keras import models, layers, optimizers
from tensorflow.python.keras.saving import hdf5_format
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
import h5py, itertools, collections
import itertools
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input

##################
# Verifications:
#################
print('GPU is used.' if len(tf.config.list_physical_devices('GPU')) > 0 else 'GPU is NOT used.')
print("Tensorflow version: " + tf.__version__)

In [ ]:
'''
    Split train and validation.
'''
# We define the size of input images to 128x128 pixels.
image_size = (256, 256)
# We define the batch size
batch_size = 32

# Create an image generator with a fraction of images reserved for validation:
image_generator = ImageDataGenerator(
                                     validation_split=0.5)

# Now, we create a training data iterator by creating batchs of images of the same size as 
# defined previously, i.e., each image is resized in a 64x64 pixels format.
train_ds =  DirectoryIterator(
    "dataset/",
    image_generator,
    class_mode='categorical',
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'training',
)

# Similarly, we create a validation data iterator by creating batchs of images of the same size as 
# defined previously, i.e., each image is resized in a 64x64 pixels format.
val_ds = DirectoryIterator(
    "dataset/",
    image_generator,
    class_mode='categorical',
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'validation',
    shuffle=False
)

# We save the list of classes (labels).
class_names = list(train_ds.class_indices.keys())

# We also save the number of labels.
num_classes = train_ds.num_classes


In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape=image_size + (3,),
                                               include_top=False,
                                               weights='imagenet')
# base_model.trainable = False


In [ ]:
# # Defining your model here:
model = models.Sequential()
# model.add(keras.Input(shape=image_size + (3,))) 
# model.add(layers.experimental.preprocessing.Rescaling(1./255))
# model.add(VGG19(input_shape=image_size + (3,), include_top=False, weights="imagenet"))

model.add(base_model)
model.add(tf.keras.layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))
# Print a summary of the model
model.summary()

base_learning_rate = 0.0001
model.compile(optimizers.Adam(learning_rate = base_learning_rate),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Start the trining by defining the number of epochs to train, the traing set and the validation set.
history = model.fit(train_ds, epochs=100,
    validation_data=val_ds,
)
